In [12]:

### Make sure the file is executed from the LFR/python directory

## Import libraries section ##
import numpy as np
import cv2
import os
import math
import re
from LFR_utils import read_poses_and_images,pose_to_virtualcamera, init_aos, init_window
import LFR_utils as utils
import pyaos
import glm
import glob
from PIL import Image
import re


In [2]:
folder_path = r'/home/mdigruber/AOS/AOS for Drone Swarms/LFR/data/img_markus'    ## Enter path to where your images are saved.
result_path = r'/home/mdigruber/AOS/AOS for Drone Swarms/LFR/data/FP'      ## Enter path to where you want the results to be saved.
set_folder = r'/home/mdigruber/AOS/AOS for Drone Swarms/LFR/python'      ## Enter path to your LFR/python directory

In [3]:
Focal_plane = 35  # Focal plane is set to the ground so it is zero.  

altitude_list = [35.01]   # Metion the altitude (all drones are in the same altitude)



In [4]:
w,h,fovDegrees = 512, 512, 35 # # resolution and field of view. This should not be changed.
render_fov = 35

if 'window' not in locals() or window == None:
                                    
    window = pyaos.PyGlfwWindow( w, h, 'AOS' )  
     
aos = pyaos.PyAOS(w,h,fovDegrees) 

aos.loadDEM( os.path.join(set_folder,'zero_plane.obj'))

In [5]:
#############################Create Poses for Initial Positions###############################################################

# Below are certain functions required to convert the poses to a certain format to be compatabile with the AOS Renderer.

def eul2rotm(theta) :
    s_1 = math.sin(theta[0])
    c_1 = math.cos(theta[0]) 
    s_2 = math.sin(theta[1]) 
    c_2 = math.cos(theta[1]) 
    s_3 = math.sin(theta[2]) 
    c_3 = math.cos(theta[2])
    rotm = np.identity(3)
    rotm[0,0] =  c_1*c_2
    rotm[0,1] =  c_1*s_2*s_3 - s_1*c_3
    rotm[0,2] =  c_1*s_2*c_3 + s_1*s_3

    rotm[1,0] =  s_1*c_2
    rotm[1,1] =  s_1*s_2*s_3 + c_1*c_3
    rotm[1,2] =  s_1*s_2*c_3 - c_1*s_3

    rotm[2,0] = -s_2
    rotm[2,1] =  c_2*s_3
    rotm[2,2] =  c_2*c_3        

    return rotm

def createviewmateuler(eulerang, camLocation):
    
    rotationmat = eul2rotm(eulerang)
    translVec =  np.reshape((-camLocation @ rotationmat),(3,1))
    conjoinedmat = (np.append(np.transpose(rotationmat), translVec, axis=1))
    return conjoinedmat

def divide_by_alpha(rimg2):
        a = np.stack((rimg2[:,:,3],rimg2[:,:,3],rimg2[:,:,3]),axis=-1)
        return rimg2[:,:,:3]/a
    
    
def pose_to_virtualcamera(vpose ):
    vp = glm.mat4(*np.array(vpose).transpose().flatten())
    #vp = vpose.copy()
    ivp = glm.inverse(glm.transpose(vp))
    #ivp = glm.inverse(vpose)
    Posvec = glm.vec3(ivp[3])
    Upvec = glm.vec3(ivp[1])
    FrontVec = glm.vec3(ivp[2])
    lookAt = glm.lookAt(Posvec, Posvec + FrontVec, Upvec)
    cameraviewarr = np.asarray(lookAt)
    #print(cameraviewarr)
    return cameraviewarr 



In [6]:
import random
# grid_resolution = 31  # Mention the same grid size that was mentioned in the simulator for simulation.

# area_size =  30    # The range is  -16.32 to 16.32 just as mentioned in the simulator for image generation 

grid_resolution = 9  # Mention the same grid size that was mentioned in the simulator for simulation.

area_size =  15    # The range is  -16.32 to 16.32 just as mentioned in the simulator for image generation 

image_posx_1 = []
image_posy_1 = []

image_posx = []
image_posy = []

grid_size= int((grid_resolution-1)/2)

numbers = [random.uniform(0.1, 0.5) for _ in range(10)]

# Below we compute the poses (x and y coordinates in metres as in simulation)
c = 0
for y in range( -grid_size,grid_size+1,1):
    for x in range(-grid_size,grid_size+1, 1):
        image_posx.append((x*area_size/grid_size))
        image_posy.append(-(y*area_size/grid_size))
        if (x*area_size/grid_size) == 0 and -(y*area_size/grid_size) == 0:
            print(c)
        c += 1

# for i in range(0, len(image_posx_1), 12):
#     image_posx.append(image_posx_1[i])

# for i in range(0, len(image_posy_1), 12):
#     image_posy.append(image_posy_1[i])
       
print('xposes:',image_posx)
print('yposes:',image_posy)
print(image_posx[len(image_posx) //2], image_posy[len(image_posx) //2])
len(image_posx)

40
xposes: [-15.0, -11.25, -7.5, -3.75, 0.0, 3.75, 7.5, 11.25, 15.0, -15.0, -11.25, -7.5, -3.75, 0.0, 3.75, 7.5, 11.25, 15.0, -15.0, -11.25, -7.5, -3.75, 0.0, 3.75, 7.5, 11.25, 15.0, -15.0, -11.25, -7.5, -3.75, 0.0, 3.75, 7.5, 11.25, 15.0, -15.0, -11.25, -7.5, -3.75, 0.0, 3.75, 7.5, 11.25, 15.0, -15.0, -11.25, -7.5, -3.75, 0.0, 3.75, 7.5, 11.25, 15.0, -15.0, -11.25, -7.5, -3.75, 0.0, 3.75, 7.5, 11.25, 15.0, -15.0, -11.25, -7.5, -3.75, 0.0, 3.75, 7.5, 11.25, 15.0, -15.0, -11.25, -7.5, -3.75, 0.0, 3.75, 7.5, 11.25, 15.0]
yposes: [15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 11.25, 11.25, 11.25, 11.25, 11.25, 11.25, 11.25, 11.25, 11.25, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 3.75, 3.75, 3.75, 3.75, 3.75, 3.75, 3.75, 3.75, 3.75, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -3.75, -3.75, -3.75, -3.75, -3.75, -3.75, -3.75, -3.75, -3.75, -7.5, -7.5, -7.5, -7.5, -7.5, -7.5, -7.5, -7.5, -7.5, -11.25, -11.25, -11.25, -11.25, -11.25, -11.25, -11.25, -11.25, -11.25, -15.0, 

81

In [7]:
with open(r'/home/mdigruber/AOS/AOS for Drone Swarms/LFR/data/poses.txt', 'r') as f:
    lines = f.readlines()
    
image_posx = []
image_posy = []
image_posz = []

for line in lines:
    image_posx.append(float(line.split(',')[0]) * -1)
    image_posy.append(float(line.split(',')[1]) * -1)
    image_posz.append(float(line.split(',')[2]) * -1)

# image_posz = image_posz * -1
print(image_posx[len(lines)//2])
print(image_posy)
print(image_posz)

-10.0
[30.0, 29.0, 28.0, 27.0, 26.0, 25.0, 24.0, 23.0, 22.0, 21.0, 20.0, 19.0, 18.0, 17.0, 16.0, 15.0, 14.0, 13.0, 12.0, 11.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, -0.0, -1.0, -2.0, -3.0, -4.0, -5.0, -6.0, -7.0, -8.0, -9.0, -10.0, -11.0, -12.0, -13.0, -14.0, -15.0, -16.0, -17.0, -18.0, -19.0, -20.0, -21.0, -22.0, -23.0, -24.0, -25.0, -26.0, -27.0, -28.0, -29.0]
[-35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0, -35.0]


In [8]:
import time

start_time = time.time()
# Below is a function to sort the image while loading from the folder
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts



## This cell generate the grid for low-resolution sapmles

In [ ]:
import shutil

data = []
for img in range(0, len(sorted(glob.glob(folder_path + '/*.png'),key=numericalSort)), 12): 
    shutil.copy(r"d:\Research\De-Blurring\codes\SOD-Pytorch\data\test\Scene_41\FS\TD_pose_"+str(img)+"_rgb.png", 
                r"d:\Research\De-Blurring\codes\SOD-Pytorch\data\test\Scene_41\FS_9\TD_pose_"+str(img)+"_rgb.png")


# Initialize a new list to store the results
# result = []
# for i, v in enumerate(range(0, 960, 31)):
#     # print(i, v)
#     if (i % 2 == 0 and i != 30):
#         result.extend(range(v, v+30, 2))  # Take 15 elements
#     if i == 15:
#         result.extend(range(v+1, v+30, 2))  # Take 15 elements
        
# result = []
# index = 0
# for i, v in enumerate(range(0, 960, 31)):
#     result.append(list(range(v, v+31)))  # Take 15 elements

# final = []
# for y in range(0, len(result), 5):
#     for x in range(0, len(result[y]), 5):
#         shutil.copy(r"d:\Research\De-Blurring\Scene_51\FS\TD_pose_"+str(result[y][x])+"_rgb.png", 
#             r"d:\Research\De-Blurring\Scene_51\FS_8\TD_pose_"+str(result[y][x])+"_rgb.png")
         
   
# for img in result: 
#     shutil.copy(r"d:\Research\De-Blurring\Scene_51\FS\TD_pose_"+str(img)+"_rgb.png", 
#                 r"d:\Research\De-Blurring\Scene_51\FS_15\TD_pose_"+str(img)+"_rgb.png")

folder_path2 = r"d:\Research\De-Blurring\codes\SOD-Pytorch\data\test\Scene_41\FS_9"
# mona = 0
# for i in result:
#     mona += 1
#     print("{:4}".format(i), end=" ")
#     if mona % 15 == 0 and mona != 0:
#         print()
# print(len(result))

In [9]:
import glob
imagelist = []
center_index = 0
check = True
for img in sorted(glob.glob(folder_path + '/*.png'),key=numericalSort):      # Enter path to the images directory which should contain 11 images.
    if "TD_pose_40_rgb" not in img and check:
        center_index += 1
    else:
        check = False
    # print(img)
    n= cv2.imread(img)
    imagelist.append(n)  
site_poses = []
print(center_index)
# Below is the poses computation from x,y coordinates to a transformation matrix to load into the renderer.
    
for i in range(len(imagelist)):   # 2 because we are only dealing with 2 images/poses for every iteration.
    #For Bottom Up comment the following 2 lines and add - to image_posey
    EastCentered = (image_posx[i] - 0.0) 
    NorthCentered = (0.0 - image_posy[i]) 
    M = createviewmateuler(np.array([0.0, 0.0, 0.0]),np.array( [image_posx[i], image_posy[i], image_posz[i] ] )) # z = -35 for all images
    #print('m',M)
    ViewMatrix = np.vstack((M, np.array([0.0,0.0,0.0,1.0],dtype=np.float32)))
    #print(ViewMatrix)
    camerapose = np.asarray(ViewMatrix.transpose(),dtype=np.float32)
    #print(camerapose)
    site_poses.append(camerapose) 

60


In [10]:
#### Note the altitude in poses is set as -35, the z axis is positive going downwards. So the focal plane value you have to set  will be in range 0 to -35 (0 for ground -35 the camera)    
    
center_index = len(imagelist) // 2 #480 Index or camera position at which you want to generate the integral image   
aos.clearViews()   # Every time you call the renderer you should use this line to clear the previous views  
for i in range(len(imagelist)):
        aos.addView(imagelist[i], site_poses[i], "DEM BlobTrack")  # Here we are adding images to the renderer one by one.

In [11]:
image_posx[len(image_posx) //2]
# print(len(image_posx))

-10.0

In [13]:
######## If you want to do focal stack of integral images just put everything below in a loop where you keep updating the Focal_plane variable        
focal = 0
fs = []
print(len(image_posx) //2)
for i in range(550):
    aos.setDEMTransform([0,0,focal])
    proj_RGBimg = aos.render(pose_to_virtualcamera(site_poses[len(image_posx) //2]), render_fov)
    #proj_RGBimg = aos.render(pose_to_virtualcamera(site_poses[center_index]), render_fov)
    tmp_RGB = divide_by_alpha(proj_RGBimg)
    # # control Contrast by 1.5 
    # alpha = 0  
    # # control brightness by 50 
    # beta = 50  
    # image2 = cv2.convertScaleAbs(tmp_RGB, alpha=alpha, beta=beta) 
    fs.append(tmp_RGB)
    #  cv2.cvtColor(tmp_RGB, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(os.path.join( result_path, f'{str(i)}.png'), tmp_RGB)   # Final result. Check the integral result in the integrals folder.
    focal -= 0.03

print(f'Time taken = {time.time() - start_time}')
# fs_copy = fs.copy()
# index = 1
# for i in (reversed(fs[1:])):
#     # x = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
#     cv2.imwrite(os.path.join( result_path, f'{str(index)}.png'), i)   # Final result. Check the integral result in the integrals folder.
#     index += 1


# index = 6
# for i in (fs):
#     # x = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
    
#     cv2.imwrite(os.path.join( result_path, f'{str(index)}.png'), i)   # Final result. Check the integral result in the integrals folder.
#     index += 1

# Not used    


# focal = -5
# for i in range(50):
#     aos.setDEMTransform([0,0,focal])
#     proj_RGBimg = aos.render(pose_to_virtualcamera(site_poses[center_index]), render_fov)
#     tmp_RGB = divide_by_alpha(proj_RGBimg)
#     cv2.imwrite(os.path.join( result_path, f'{str(i)}.png'), tmp_RGB)   # Final result. Check the integral result in the integrals folder.
#     focal += 0.01
    
# focal = 0
# for i in range(50, 100):
#     aos.setDEMTransform([0,0,focal])
#     proj_RGBimg = aos.render(pose_to_virtualcamera(site_poses[center_index]), render_fov)
#     tmp_RGB = divide_by_alpha(proj_RGBimg)
#     cv2.imwrite(os.path.join( result_path, f'{str(i)}.png'), tmp_RGB)   # Final result. Check the integral result in the integrals folder.
#     focal -= 0.01

30
Time taken = 21.294007301330566


In [31]:

del window

del aos

In [40]:
image1 = cv2.imread(r"d:\Research\De-Blurring\Scene_51\FP_8\33.png")
image2 = cv2.imread(r"d:\Research\De-Blurring\Scene_51\FP_16\33.png")
image3 = cv2.imread(r"d:\Research\De-Blurring\Scene_51\FP_31\33.png")


image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
image3 = cv2.cvtColor(image3, cv2.COLOR_BGR2GRAY)
# print(image1[50][50])
# print(image2[50][50])
# print(image3[50][50])

matrix = [[0 for _ in range(960)] for _ in range(960)]

for i in range(960):
    for j in range(960):
        vectors = []
        vectors.append(image1[i][j])
        vectors.append(image2[i][j])
        vectors.append(image3[i][j])
        vectors = np.array(vectors)
        
        matrix[i][j] = np.max(vectors - np.mean(vectors))

# image1 = (image1 - image1.min()) / (image1.max() - image1.min())
# image2 = (image2 - image2.min()) / (image2.max() - image2.min())

# Convert images to float32 for accurate subtraction
# image1_float = image1.astype(np.float32)
# image2_float = image2.astype(np.float32)

# # Perform element-wise subtraction
# subtracted_image = np.abs(image1_float - image2_float)

# Convert the result back to uint8 format
# subtracted_image = subtracted_image.astype(np.uint8)
from PIL import Image

pixel_values = [[int(value) * 15 for value in row] for row in matrix]

image = Image.new('L', (len(matrix[0]), len(matrix)))
image.putdata([pixel for row in pixel_values for pixel in row])

# Save the image
image.save(r'd:\Research\De-Blurring\Scene_51\matrix_image.png')



# If you want to save the result
# cv2.imwrite(r"d:\Research\De-Blurring\Scene_51\333.png", subtracted_image * 255)

[ WARN:0@209.254] global loadsave.cpp:241 findDecoder imread_('d:\Research\De-Blurring\Scene_51\FP_8\33.png'): can't open/read file: check file path/integrity
[ WARN:0@209.255] global loadsave.cpp:241 findDecoder imread_('d:\Research\De-Blurring\Scene_51\FP_16\33.png'): can't open/read file: check file path/integrity
[ WARN:0@209.255] global loadsave.cpp:241 findDecoder imread_('d:\Research\De-Blurring\Scene_51\FP_31\33.png'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
image1 = cv2.imread(r"d:\Research\De-Blurring\Scene_1\inverted_mask\TD38_pose_0_rgb.png")
image2 = cv2.imread(r"d:\Research\De-Blurring\Scene_1\6.png")

image1 = image1[2:-2, 2:-2]

# image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
# image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

image1 = (image1 - image1.min()) / (image1.max() - image1.min())
image2 = (image2 - image2.min()) / (image2.max() - image2.min())

cv2.imwrite(r"d:\Research\De-Blurring\Scene_1\sss.png", abs(image1 - image2) * 255)
# cv2.imwrite(r"d:\Research\De-Blurring\Scene_1\sss.png", image1 * image2 * 255)